In [1]:
import itertools as it
import datetime as dt

import numpy as np

import pandas as pd

%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import (
    KFold, StratifiedKFold, train_test_split)
from sklearn.feature_selection import RFECV

from sklearn.svm import SVC

import joblib

from libs.container import Container
import dataset

In [2]:
cpu = joblib.cpu_count()

In [3]:
names = ["b261", "b277", "b278", "b360"]
combs = list(it.combinations(names, 2))
combs

[('b261', 'b277'),
 ('b261', 'b278'),
 ('b261', 'b360'),
 ('b277', 'b278'),
 ('b277', 'b360'),
 ('b278', 'b360')]

In [4]:
_, sdata, _ = dataset.load_tile_clf()



# del sdata

In [5]:
BEST_PARAMS = joblib.load("results/hp_selection.pkl.bz2")
SVC_LINEAR_PARAMS = BEST_PARAMS["svc_linear"].best_params_

del BEST_PARAMS

SVC_LINEAR_PARAMS

{'C': 100, 'kernel': 'linear', 'probability': True}

In [ ]:
res = {}
for t0, t1 in combs:
    print(dt.datetime.now())
        
    df = pd.concat([sdata[t0], sdata[t1]])

    cls = {name: idx for idx, name in enumerate(df.tile.unique())}
    print(cls)
    df["cls"] = df.tile.apply(cls.get)

    X = df[dataset.FEATURES].values
    y = df.cls.values
    
    clf = SVC(**SVC_LINEAR_PARAMS)
    sel = RFECV(clf, n_jobs=-1, cv=5)
    
    print("fit")
    sel.fit(X, y)
    
    print("storing")
    res[(t0, t1)] = sel
    
    print("-----------------------------")
    

2020-08-06 00:44:03.181420
{'b261': 0, 'b277': 1}
fit


In [ ]:
joblib.dump(res, "results/rfecv.pkl.bz2", compress=3)